In [1]:
import google.generativeai as genai

from config import env
from questions_mysql import QuestionsMysql


class GeminiAIAPI(object):

    def __init__(self):
        self._client = genai.configure(api_key=env.str("GEMINI_API_KEY"))
    
    def ask_gemini(self, question, model_str):
        res = ""
        model = genai.GenerativeModel(model_str)
        response = model.generate_content(question)
        #print(response.text)

        for chunk in response:
            print(chunk.text)
            res += chunk.text
        return res

if __name__ == '__main__':
    api = GeminiAIAPI()
    
    qsql = QuestionsMysql()
    question_dict = qsql.get_questions()
    
    response = "\n"
    # Prompt the questions in a batch of 20 
    for startIndex in range (1, len(question_dict) + 1, 1):
        prompt_str = qsql.get_prompt_string(question_dict, startIndex, 1)
        response += api.ask_gemini(prompt_str, 'gemini-1.5-pro')

   
    score = qsql.get_score(response, question_dict)
    print("\n")
    print('Gemini 1.5 Pro :' + score)
    


/Users/mohanqi/vscode/ai/ai-benchmark/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.C
2.A
3.A
4.D
5.A 

6.B
7.A
8.D
9.B
10.D 

11.B
12.A
13.C
14.D
15.B 

16.A
17.B
18.B
19.A
20.C 

21.B
22.D
23.B
24.B
25.A 

26.B
27.A
28.A
29.D
30.A 

31.A
32.D
33.A
34.A
35.C 

36.B
37.D
38.C
39.D
40.A 

41.B
42.C
43.C
44.D
45.D 

46.C
47.D
48.D
49.D
50.D 

51.C
52.B
53.B
54.C
55.A 

56.A
57.A
58.A
59.B
60.D 

61.A
62.C
63.D
64.D
65.C 

66.D
67.A
68.A
69.A
70.D 

71.B
72.B
73.D
74.A
75.C 

76.C
77.C
78.D
79.B
80.D 

81.C
82.A
83.C
84.A
85.B 

86.A
87.C
88.B
89.D
90.A 

91.D
92.D
93.B
94.C
95.C 

96.D
97.C
98.B
99.D
100.C 

101.B
102.C
103.D
104.D
105.D 

106.A
107.A
108.C
109.C
110.A 

111.C
112.C
113.A
114.C
115.D 

116.B
117.C
118.B
119.D
120.B 

121.D
122.D
123.B
124.B
125.D 

126.D
127.A
128.A
129.D
130.B 

131.B
132.C
133.D
134.C
135.D 

136.C
137.D
138.B
139.A
140.D 

141.B
142.B
143.C
144.C
145.D 

146.B
147.D
148.A
149.D
150.B 

151.A
152.D
153.D
154.B
155.D 

156.C
157.B
158.A
159.A
160.A 

161.D
162.B
163.A
164.D
165.A 

166.B
167.C
168.B
169.C
170.B 

171.C
172.A
173.C
17